# Patient_info, 2

## Library, Data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
from dfply import *

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host = "localhost",
    user = "postgres",
    password="June3060!!",
    database = "mimic_data",
    port="5432"
)

cursor = conn.cursor()
print("database connected")

In [369]:
df = pd.read_csv("../Data/paired_df.csv", index_col="Unnamed: 0")

In [370]:
df2 = df.copy()

## info
* COPD → Chronic Obstructive Pulmonary Disease
* gender
* age
* height
* Weight
* BP
* GCS (fin)
* SAPs ll (fin)
* OASIS (fin)

### COPD

* COPD 여부를 넣는 건지..?

In [75]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_hosp.drgcodes;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
drgcodes = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  1.9793009757995605


In [76]:
drgcodes.head()

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,10000032,22595853,APR,283,OTHER DISORDERS OF THE LIVER,2.0,2.0
1,10000032,22595853,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
2,10000032,22841357,APR,279,HEPATIC COMA & OTHER MAJOR ACUTE LIVER DISORDERS,3.0,2.0
3,10000032,22841357,HCFA,442,"DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...",NaN,NaN
4,10000032,25742920,APR,283,OTHER DISORDERS OF THE LIVER,3.0,2.0


In [77]:
copd_drgcodes = drgcodes[drgcodes["description"].str.contains("CHRONIC OBSTRUCTIVE PULMONARY DISEASE")]
copd_drgcodes

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
93,10001884,24746267,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,3.0,3.0
94,10001884,24746267,HCFA,191,CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC,NaN,NaN
95,10001884,24962904,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,2.0,2.0
96,10001884,24962904,HCFA,192,CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/O CC/MCC,NaN,NaN
99,10001884,26184834,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,2.0,2.0
...,...,...,...,...,...,...,...
603831,19991135,29511669,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,2.0,2.0
603832,19991135,29511669,HCFA,191,CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC,NaN,NaN
604237,19998330,24096838,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,4.0,4.0
604241,19998330,27282608,APR,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,3.0,2.0


In [78]:
copd_drgcodes["drg_code"].value_counts()

drg_code
140    2742
191    1081
190    1016
192     572
Name: count, dtype: int64

In [79]:
# copd...?
copd_drgcodes[copd_drgcodes["subject_id"] == 10004401]

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality


### Patients → gender, age

In [80]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_hosp.patients;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
patients = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

patients

경과 시간 :  0.7239112854003906


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000068,F,19,2160,2008 - 2010,None
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,None
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,None
299708,19999829,F,28,2186,2008 - 2010,None
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,None


In [91]:
patients2 = patients[["subject_id", "gender", "anchor_age", "anchor_year"]]
patients2

,subject_id,gender,anchor_age,anchor_year
0,10000032,F,52,2180
1,10000048,F,23,2126
2,10000068,F,19,2160
3,10000084,M,72,2160
4,10000102,F,27,2136
...,...,...,...,...
299707,19999828,F,46,2147
299708,19999829,F,28,2186
299709,19999840,M,58,2164
299710,19999914,F,49,2158


In [92]:
df >> left_join(patients2, by="subject_id")

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,gender,anchor_age,anchor_year
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,F,68,2122
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,F,72,2134
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,M,82,2141
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,M,82,2141
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,M,82,2141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,F,63,2126
9392,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,F,63,2126
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,M,63,2161
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,M,41,2146


----------

### omr → BP / Weight / Height

In [512]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_hosp.omr;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
omr = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)

경과 시간 :  11.918288946151733


In [513]:
omr

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
1,10000032,2180-04-27,1,Weight (Lbs),94
2,10000032,2180-05-07,1,BMI (kg/m2),18.0
3,10000032,2180-05-07,1,Height (Inches),60
4,10000032,2180-05-07,1,Weight (Lbs),92.15
...,...,...,...,...,...
6439164,19999828,2148-02-26,1,Blood Pressure,115/79
6439165,19999828,2148-04-29,1,Blood Pressure,105/67
6439166,19999828,2148-07-22,1,Blood Pressure,104/76
6439167,19999828,2148-10-19,1,Blood Pressure,112/73


* Weight 값 / Height 값이 두개로 나뉘어져있음, 확인 필요
* Blood Pressure, Sitting, Lying, Standing... 일단 Blood Pressure 우선으로 진행

In [514]:
omr["result_name"].value_counts()

result_name
Blood Pressure                      2169549
Weight (Lbs)                        1889542
BMI (kg/m2)                         1662112
Height (Inches)                      706906
Blood Pressure Sitting                 3400
Blood Pressure Lying                   2764
Blood Pressure Standing (1 min)        2560
Blood Pressure Standing (3 mins)        626
BMI                                     554
Blood Pressure Standing                 523
Weight                                  354
eGFR                                    240
Height                                   39
Name: count, dtype: int64

### Height

* Height 값이 inches가 생략이 된 것 같다 (64, 71 등.. 값이 inches 단위가 Reasonable한 듯)
* Height (Inches) 로 통일

In [515]:
omr[omr["result_name"] == "Height"]

,subject_id,chartdate,seq_num,result_name,result_value
98063,10152950,2180-12-04,1,Height,64.0
246085,10391615,2116-10-22,1,Height,64.50
246087,10391615,2116-10-29,1,Height,64.50
304771,10495326,2138-11-20,1,Height,69.50
336955,10550602,2153-12-30,1,Height,71.
461757,10738205,2174-09-12,1,Height,62.50
799047,11266244,2154-01-28,1,Height,61.0
972593,11540283,2200-07-06,1,Height,64.0
1070847,11692388,2182-07-11,1,Height,60.50
1256314,11985668,2186-03-01,1,Height,68.50


In [516]:
omr.loc[omr["result_name"] == "Height", "result_name"] = "Height (Inches)"

In [517]:
omr_heights = omr[omr["result_name"] == "Height (Inches)"]
omr_heights

,subject_id,chartdate,seq_num,result_name,result_value
3,10000032,2180-05-07,1,Height (Inches),60
29,10000032,2180-06-22,1,Height (Inches),60
42,10000084,2160-11-21,1,Height (Inches),70
55,10000117,2175-09-08,1,Height (Inches),64.5
72,10000117,2176-09-08,1,Height (Inches),65
...,...,...,...,...,...
6439117,19999784,2121-06-20,1,Height (Inches),67.3
6439124,19999784,2121-08-18,1,Height (Inches),66.50
6439128,19999784,2121-08-25,1,Height (Inches),67
6439147,19999784,2122-02-01,1,Height (Inches),67.5


In [519]:
omr_heights["result_value"] = pd.to_numeric(omr_heights["result_value"], errors="coerce")

C:\Users\user\AppData\Local\Temp\ipykernel_52484\2609385154.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_heights["result_value"] = pd.to_numeric(omr_heights["result_value"], errors="coerce")


In [520]:
omr_heights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 706945 entries, 3 to 6439158
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   subject_id    706945 non-null  int64  
 1   chartdate     706945 non-null  object 
 2   seq_num       706945 non-null  int64  
 3   result_name   706945 non-null  object 
 4   result_value  706945 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 32.4+ MB


In [521]:
omr_heights.loc[:, "result_value_cm"] = round(omr_heights["result_value"] * 2.54, 1)

C:\Users\user\AppData\Local\Temp\ipykernel_52484\2620081647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_heights.loc[:, "result_value_cm"] = round(omr_heights["result_value"] * 2.54, 1)


In [523]:
omr_heights.head()

,subject_id,chartdate,seq_num,result_name,result_value,result_value_cm
3,10000032,2180-05-07,1,Height (Inches),60.0,152.4
29,10000032,2180-06-22,1,Height (Inches),60.0,152.4
42,10000084,2160-11-21,1,Height (Inches),70.0,177.8
55,10000117,2175-09-08,1,Height (Inches),64.5,163.8
72,10000117,2176-09-08,1,Height (Inches),65.0,165.1


In [524]:
# ... 평균 값을 써야하나....
omr_heights[omr_heights["subject_id"] == 19999784]

,subject_id,chartdate,seq_num,result_name,result_value,result_value_cm
6438979,19999784,2119-06-18,1,Height (Inches),67.00,170.2
6438988,19999784,2119-08-28,1,Height (Inches),67.00,170.2
6438992,19999784,2119-09-03,1,Height (Inches),67.00,170.2
6439003,19999784,2119-09-19,1,Height (Inches),66.50,168.9
6439016,19999784,2119-10-16,1,Height (Inches),67.00,170.2
6439019,19999784,2119-10-17,1,Height (Inches),66.50,168.9
6439033,19999784,2119-12-30,1,Height (Inches),66.75,169.5
6439042,19999784,2120-02-13,1,Height (Inches),66.50,168.9
6439051,19999784,2120-04-15,1,Height (Inches),66.50,168.9
6439060,19999784,2120-06-24,1,Height (Inches),67.00,170.2


### Weights

In [24]:
omr["result_name"].value_counts()

result_name
Blood Pressure                      2169549
Weight (Lbs)                        1889542
BMI (kg/m2)                         1662112
Height (Inches)                      706945
Blood Pressure Sitting                 3400
Blood Pressure Lying                   2764
Blood Pressure Standing (1 min)        2560
Blood Pressure Standing (3 mins)        626
BMI                                     554
Blood Pressure Standing                 523
Weight                                  354
eGFR                                    240
Name: count, dtype: int64

In [25]:
omr["result_name"].value_counts()

result_name
Blood Pressure                      2169549
Weight (Lbs)                        1889542
BMI (kg/m2)                         1662112
Height (Inches)                      706945
Blood Pressure Sitting                 3400
Blood Pressure Lying                   2764
Blood Pressure Standing (1 min)        2560
Blood Pressure Standing (3 mins)        626
BMI                                     554
Blood Pressure Standing                 523
Weight                                  354
eGFR                                    240
Name: count, dtype: int64

In [26]:
omr.loc[omr["result_name"] == "Weight", "result_name"] = "Weight (Lbs)"

In [27]:
omr_weights = omr[omr["result_name"] == "Weight (Lbs)"]
omr_weights

,subject_id,chartdate,seq_num,result_name,result_value
1,10000032,2180-04-27,1,Weight (Lbs),94
4,10000032,2180-05-07,1,Weight (Lbs),92.15
5,10000032,2180-05-07,2,Weight (Lbs),92.15
6,10000032,2180-05-07,3,Weight (Lbs),92.15
7,10000032,2180-05-07,4,Weight (Lbs),92.15
...,...,...,...,...,...
6439144,19999784,2122-01-23,1,Weight (Lbs),174.9
6439148,19999784,2122-02-01,1,Weight (Lbs),173
6439151,19999784,2122-02-21,1,Weight (Lbs),173.4
6439154,19999784,2122-03-29,1,Weight (Lbs),173.9


In [28]:
omr_weights["result_value"] = pd.to_numeric(omr_weights["result_value"], errors="coerce")

C:\Users\user\AppData\Local\Temp\ipykernel_52484\1030577628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_weights["result_value"] = pd.to_numeric(omr_weights["result_value"], errors="coerce")


In [29]:
omr_weights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1889896 entries, 1 to 6439159
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   subject_id    int64  
 1   chartdate     object 
 2   seq_num       int64  
 3   result_name   object 
 4   result_value  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 86.5+ MB


In [30]:
omr_weights.loc[:, "result_value_kg"] = round(omr_weights["result_value"] / 2.205, 2) 

C:\Users\user\AppData\Local\Temp\ipykernel_52484\2330733571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omr_weights.loc[:, "result_value_kg"] = round(omr_weights["result_value"] / 2.205, 2)


In [31]:
omr_weights

,subject_id,chartdate,seq_num,result_name,result_value,result_value_kg
1,10000032,2180-04-27,1,Weight (Lbs),94.00,42.63
4,10000032,2180-05-07,1,Weight (Lbs),92.15,41.79
5,10000032,2180-05-07,2,Weight (Lbs),92.15,41.79
6,10000032,2180-05-07,3,Weight (Lbs),92.15,41.79
7,10000032,2180-05-07,4,Weight (Lbs),92.15,41.79
...,...,...,...,...,...,...
6439144,19999784,2122-01-23,1,Weight (Lbs),174.90,79.32
6439148,19999784,2122-02-01,1,Weight (Lbs),173.00,78.46
6439151,19999784,2122-02-21,1,Weight (Lbs),173.40,78.64
6439154,19999784,2122-03-29,1,Weight (Lbs),173.90,78.87


In [32]:
omr_weights["result_value_kg"].isna().sum()

0

### BP

In [33]:
omr["result_name"].value_counts()

result_name
Blood Pressure                      2169549
Weight (Lbs)                        1889896
BMI (kg/m2)                         1662112
Height (Inches)                      706945
Blood Pressure Sitting                 3400
Blood Pressure Lying                   2764
Blood Pressure Standing (1 min)        2560
Blood Pressure Standing (3 mins)        626
BMI                                     554
Blood Pressure Standing                 523
eGFR                                    240
Name: count, dtype: int64

In [34]:
omr_bp = omr[omr["result_name"].str.contains("Blood")]
omr_bp

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-04-27,1,Blood Pressure,110/65
21,10000032,2180-05-25,1,Blood Pressure,106/60
24,10000032,2180-06-01,1,Blood Pressure,121/77
27,10000032,2180-06-22,1,Blood Pressure,100/60
33,10000032,2180-06-30,1,Blood Pressure,102/60
...,...,...,...,...,...
6439164,19999828,2148-02-26,1,Blood Pressure,115/79
6439165,19999828,2148-04-29,1,Blood Pressure,105/67
6439166,19999828,2148-07-22,1,Blood Pressure,104/76
6439167,19999828,2148-10-19,1,Blood Pressure,112/73


In [35]:
omr_bp[omr_bp["result_name"] == "Blood Pressure Sitting"]

,subject_id,chartdate,seq_num,result_name,result_value
3384,10002804,2153-12-07,1,Blood Pressure Sitting,125/64
3392,10002804,2154-03-01,1,Blood Pressure Sitting,147/68
3402,10002804,2154-09-12,1,Blood Pressure Sitting,153/79
3469,10002807,2153-03-10,1,Blood Pressure Sitting,144/87
5956,10006029,2170-01-23,1,Blood Pressure Sitting,112/58
...,...,...,...,...,...
6434175,19991501,2177-03-17,1,Blood Pressure Sitting,155/112
6434216,19991501,2177-04-16,1,Blood Pressure Sitting,108/68
6436715,19995012,2161-05-14,1,Blood Pressure Sitting,148/67
6436753,19995012,2162-01-10,1,Blood Pressure Sitting,110/70


In [112]:
omr_bp[omr_bp["subject_id"] == 10001884]

,subject_id,chartdate,seq_num,result_name,result_value
1089,10001884,2124-07-26,1,Blood Pressure,120/80
1091,10001884,2125-01-24,1,Blood Pressure,140/70
1093,10001884,2125-07-28,1,Blood Pressure,130/80
1095,10001884,2125-11-03,1,Blood Pressure,160/70
1097,10001884,2125-11-28,1,Blood Pressure,150/50
1099,10001884,2126-01-19,1,Blood Pressure,140/60
1101,10001884,2126-02-09,1,Blood Pressure,138/60
1105,10001884,2126-04-19,1,Blood Pressure,160/88
1108,10001884,2126-06-30,1,Blood Pressure,120/70
1111,10001884,2126-07-13,1,Blood Pressure,148/76


In [110]:
df

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN
9392,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN


### GCS

* Glasgow Coma Scale
    * 중추신경계의 기능을 평가하는 데 사용, 의식 수준 측정하는 데 주로 사용
    * 뇌 손상 정도를 판단하고, 환자의 의식 상태를 객관적으로 평가하는데 도움이 된다.
    * 눈의 열림 정도, 언어 반응, 근육 운동 기능을 기반 / 3가지 지표에 대한 점수 합산

In [36]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_derived.gcs;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
gcs = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)
gcs

경과 시간 :  3.755841016769409


,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [407]:
df2 = df.copy()

In [414]:
df2.head()

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN


In [494]:
def group_df_gcs(group):
    # group의 index, row를 하나씩 확인
    for idx, row in group.iterrows():

        # 만약, int_stayid가 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # gcs의 subject_id, stay_id가 row의 subject_id, int_stayid를 만족하는 데이터 프레임을 만든다.
            gcs_df = gcs[(gcs["subject_id"] == row["subject_id"]) & (gcs["stay_id"] == row["int_stayid"])]

            # row의 intubationtime보다 작거나 같은 gcs_df의 charttime 중 가장 마지막 index를 반환
            suitable_location = gcs_df[gcs_df["charttime"] <= row["intubationtime"]].last_valid_index()
            try:
                # 반환된 index를 이용하여 값 할당
                group.loc[idx, "gcs"] = gcs_df.at[suitable_location, "gcs"]
            except:
                # 만약 반환된 index가 없다면 (intubationtime이 가장 최소 값이라면)
                try:
                    # gcs_df의 charttime이 가장 작은 값을 index로 가지는 gcs 값 할당
                    group.loc[idx, "gcs"] = gcs_df.at[gcs_df["charttime"].idxmin(), "gcs"]
                except:
                    # 그래도 값이 없다면 nan 값 할당
                    group.loc[idx, "gcs"] = np.nan
        # 만약 int_stayid가 null 값이라면                    
        else:

            # gcs의 subject_id, stay_id가 row의 subject_id, ext_stayid를 만족하는 데이터 프레임을 만든다.
            gcs_df = gcs[(gcs["subject_id"] == row["subject_id"]) & (gcs["stay_id"] == row["ext_stayid"])]

            # row의 extubationtime보다 크거나 같은 gcs_df 의 charttime 중 가장 최근 index를 반환
            suitable_location = gcs_df[gcs_df["charttime"] >= row["extubationtime"]].first_valid_index()
            try:
                # 반환된 index를 이용하여 값 할당
                group.loc[idx, "gcs"] = gcs_df.at[suitable_location, "gcs"]
            except:
                # 만약 반환된 index가 없다면 (extubationtime이 최대 값이라면)
                try:
                    # gcs_df의 charttime이 가장 큰 값을 index로 가지는 gcs 값 할당
                    group.loc[idx, "gcs"] = gcs_df.at[gcs_df["charttime"].idxmax(), "gcs"]
                except:
                    # 그래도 값이 없다면 nan 값 할당
                    group.loc[idx, "gcs"] = np.nan
    # group return
    return group

In [495]:
df_gcs = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_gcs)

In [496]:
df_gcs

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,gcs
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,15.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,7.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,15.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,9.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,15.0
9392,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,14.0
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,14.0
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,10.0


### Oasis

* 미국에서 홈 헬스케어 환자의 평가 및 결과를 기록하는 데 사용되는 평가 도구
* 홈 헬스케어 서비스에서 사용, 환자의 건강상태, 기능 능력, 사회적 지원 등을 평가하여 개별 케어 플랜 개발하는데 도움
* 측정 항목 : 숨 쉬기, 식사, 식사 후 상태, 대변/방뇨, 침상/의자 이동, 대개인위생 등의 항목에 대한 평가 포함

In [37]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_derived.oasis;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
oasis = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)
oasis

경과 시간 :  0.628617525100708


,subject_id,hadm_id,stay_id,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,...,resprate,resp_rate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,15726063,23069026,36415879,36,0.17014499605424941551,53.3839872681065644,6,73.0000000000000000,3,15.0,...,31.0,6.0,38.56,2.0,4580.0,0.0,1,9,0,6
1,14304873,20156152,35621403,28,0.06884295828308623481,75.1505757369999434,6,9241.9666666666666667,1,14.0,...,32.0,6.0,35.40,4.0,2390.0,1.0,0,0,0,6
2,12117555,25128945,37212487,34,0.13709860673818602936,82.37789823064038515712,9,14055.783333333333,1,9.0,...,27.0,1.0,36.6150000000000000,0.0,85.0,10.0,0,0,0,6
3,16645325,26608451,37532286,33,0.12270071663757324864,77.62581604951116124530,9,1691.9833333333333333,1,15.0,...,24.0,1.0,38.00,2.0,1015.0,5.0,0,0,0,6
4,10989579,26999802,38267080,26,0.05418719720382744723,25.87166522470033566786,3,95.0000000000000000,3,15.0,...,34.0,6.0,36.39,2.0,1225.0,5.0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16312808,28382354,37548942,13,0.01080267545727155188,39.94355629661673325874,3,1130.9166666666666667,0,15.0,...,24.0,1.0,39.61,2.0,3575.0,0.0,0,0,0,6
73177,14472165,20369424,38998125,33,0.12270071663757324864,57.95088210921343474555,6,4255.2333333333333333,1,14.0,...,44.0,6.0,36.22,2.0,2670.0,0.0,0,0,0,6
73178,19603646,24079072,35813245,28,0.06884295828308623481,47.62103231099745739355,3,5557.1833333333333333,1,15.0,...,26.0,1.0,36.39,2.0,1125.0,5.0,1,9,0,6
73179,16915170,20987661,38612688,20,0.02596747254482635520,47.84390204911325154890,3,23.0000000000000000,3,13.0,...,17.5,0.0,36.94,2.0,1500.0,1.0,0,0,0,6


In [499]:
len(oasis), oasis["stay_id"].nunique()

(73181, 73181)

In [505]:
df2 = df.copy()

In [506]:
def group_df_oasis(group):

    # group의 index, row를 하나씩 할당
    for idx, row in group.iterrows():

        # oasis의 subject_id, hadm_id가 row의 subject_id, hadm_id와 동일한 oasis_df를 만든다.
        oasis_df = oasis[(oasis["subject_id"] == row["subject_id"]) & (oasis["hadm_id"] == row["hadm_id"])]

        # 만약 row의 int_stayid가 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # oasis_df의 stay_id가 row의 int_stayid를 만족하는 index를 찾는다
            # oasis 데이터 프레임에서 stay_id는 중복된 값이 없고, 전부 unique 값을 갖는다.
            oasis_idx = oasis_df[oasis_df["stay_id"] == row["int_stayid"]].first_valid_index()

            # group의 해당 index에 할당
            group.loc[idx, "oasis"] = oasis_df.loc[oasis_idx, "oasis"]

        # 만약 row의 int_stayid가 null 값이라면, ext_stayid로 대체.
        else:
            oasis_idx = oasis_df[oasis_df["stay_id"] == row["ext_stayid"]].first_valid_index()
            group.loc[idx, "oasis"] = oasis_df.loc[oasis_idx, "oasis"]
    return group

In [507]:
df_oasis = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_oasis)
df_oasis

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,oasis
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,34.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,60.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,53.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,46.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,43.0
9392,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,43.0
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,46.0
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,25.0


### saps ii

* 중증 환자의 병리 생리학적 상태를 평가, 초기 사망 위험을 예측하는 데 사용되는 평가도구
* 주로 중환자실 or 응급실에서 중증 환자의 초기 평가에 활용
* 심장, 호흡, 혈압, 체온, 혈액 가스 및 혈액 pH 등을 평가
* 환자의 나이와 기존 질병 유무 등을 고려하여 점수 계산

In [38]:
start_time = time.time()

cursor = conn.cursor()
cursor.execute("select * from mimiciv_derived.sapsii;")
rows = cursor.fetchall()
columns_name = [desc[0] for desc in cursor.description]
sapsii = pd.DataFrame(rows, columns=columns_name)

end_time = time.time()
wait_time = end_time - start_time
print("경과 시간 : ", wait_time)
sapsii

경과 시간 :  0.4143095016479492


,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,14046553,20000094,35605481,2150-03-02 15:19:31,2150-03-03 15:19:31,98,0.982729,18,11.0,13.0,...,11.0,10.0,0.0,0.0,0.0,3.0,0.0,26.0,0,6
1,14990224,20000147,30503572,2121-08-30 16:33:54,2121-08-31 16:33:54,35,0.166523,15,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
2,17913090,20000351,30593599,2145-06-13 20:10:27,2145-06-14 20:10:27,24,0.058153,7,0.0,0.0,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,6
3,16788749,20000808,30610654,2180-01-17 19:32:10,2180-01-18 19:32:10,17,0.025876,7,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
4,16788749,20000808,35191063,2180-01-12 22:07:00,2180-01-13 22:07:00,26,0.071716,7,0.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,14641622,29999186,32199257,2135-07-12 01:51:00,2135-07-13 01:51:00,30,0.106398,7,0.0,5.0,...,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0,8
73177,10053207,29999444,31112026,2199-12-14 19:52:26,2199-12-15 19:52:26,26,0.071716,7,4.0,0.0,...,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0,6
73178,13478841,29999498,33874605,2188-05-13 12:03:00,2188-05-14 12:03:00,34,0.152870,16,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,9,6
73179,10355856,29999625,36975675,2157-11-07 11:51:00,2157-11-08 11:51:00,32,0.128048,12,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0,8


In [500]:
len(sapsii), sapsii["stay_id"].nunique()

(73181, 73181)

In [503]:
df2 = df.copy()

In [501]:
def group_df_sapsii(group):

    # group의 index,와 row를 하나씩 가져온다.
    for idx, row in group.iterrows():

        # sapsii의 subject_id, hadm_id가 row의 subject_id, hadm_id를 각각 만족하는 sapsii_df를 만든다
        sapsii_df = sapsii[(sapsii["subject_id"] == row["subject_id"]) & (sapsii["hadm_id"] == row["hadm_id"])]

        # row의 int_stayid가 만약 null 값이 아니라면
        if pd.notnull(row["int_stayid"]):

            # sapsii_df의 stay_id와 row의 int_stayid가 똑같은 값을 찾는다.
            sapsii_idx = sapsii_df[sapsii_df["stay_id"] == row["int_stayid"]].first_valid_index()

            # 해당 idx에 할당한다.
            group.loc[idx, "sapsii"] = sapsii_df.loc[sapsii_idx, "sapsii"]
        else:
            sapsii_idx = sapsii_df[sapsii_df["stay_id"] == row["ext_stayid"]].first_valid_index()
            group.loc[idx, "sapsii"] = sapsii_df.loc[sapsii_idx, "sapsii"]
    return group

In [504]:
df_sapsii = df2.groupby(["subject_id", "hadm_id"], group_keys=False).apply(group_df_sapsii)
df_sapsii

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime,marker,sapsii
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,NaN,44.0
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,NaN,83.0
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,NaN,NaN,NaN,NaN,2144-04-13 17:31:00,NaN,NaN,43.0
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaN,NaN,65.0
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,NaN,NaN,NaN,NaN,2144-02-06 11:45:00,NaN,NaN,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9391,19997367,20617667,35616526.0,2126-04-20 07:15:00,2126-05-05 18:20:00,224385.0,59.0,35616526.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,55.0
9392,19997367,20617667,NaN,2126-04-20 07:15:00,NaN,NaN,NaN,35616526.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,2126-05-19 14:15:00,NaN,NaN,55.0
9393,19999068,21606769,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,55.8,30143796.0,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2161-09-02 19:00:00,NaN,NaN,44.0
9394,19999442,26785317,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,107.5,32336619.0,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,2148-12-04 16:25:00,NaN,NaN,20.0
